## WFP-01-02-02 Landsat-8 reflectances and vegetation indices

As a land cover change analyst I want to derive the vegetation indices (NDVI, NDWI, MNDWI, NDBI) from Landsat-8 products.

### <a name="service">Service Definition

In [118]:
service = dict([('title', 'WFP-01-02-02 Landsat-8 reflectances and vegetation indices'),
                ('abstract', 'WFP-01-02-02 Landsat-8 reflectances and vegetation indices'),
                ('id', 'ewf-wfp-01-02-02')])

### <a name="parameter">Parameter Definition 

**None**


### <a name="runtime">Runtime parameter definition

**Downloaded product**

This is the downloaded Landsat-8 product reference path 

In [119]:
downloaded_reference = '/workspace/data/LC81540332017216LGN00.tar'#'LC08_L1TP_154033_20170804_20170812_01_T1'

**Input reference**

This is the Landsat-8 catalogue reference

In [120]:
input_reference = 'https://catalog.terradue.com/landsat8/search?format=atom&uid=LC08_L1TP_154033_20170804_20170812_01_T1'

### <a name="workflow">Workflow

#### Import the packages required for processing the Landsat-8 vegetation indices

In [121]:
import os
import sys
import glob
import tarfile

sys.path.append('/opt/OTB-6.2.0/lib/python')
sys.path.append('/opt/OTB-6.2.0/lib/libfftw3.so.3')
os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
os.environ['LD_LIBRARY_PATH'] = '/opt/OTB-6.2.0/lib'
os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB-6.2.0/lib/otb/applications'
os.environ['GDAL_DATA'] = '/opt/anaconda/share/gdal/'
import otbApplication

import cioppy

import lxml.etree as etree

from shapely.wkt import loads

import gdal
import osr

from shapely.geometry import box

import numpy as np


In [122]:
def get_attribute(mtl_path, attribute):
    
    mtl = open(mtl_path, 'r')
    attribs = mtl.readlines()
        
    for line in attribs:
        
        if attribute + ' = ' in line:
            return line.split('=')[1].rstrip().strip(' ').strip('""')
      

In [123]:
def offset(band):
    
    return get_attribute(ls8_metadata, 'REFLECTANCE_ADD_BAND_' + str(band))

In [124]:
def gain(band):
    
    return get_attribute(ls8_metadata, 'REFLECTANCE_MULT_BAND_' + str(band))

In [125]:
def dn_to_reflectance(band, metadata):
    
    output_name = '%s_R%s.TIF' % (get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID'),
                                                     str(band))
    
    metadata_name = '%s_R%s.xml' % (get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID'),
                                                     str(band))
    ls8_gain = gain(band)
    ls8_offset = offset(band)
    ls8_band_filename = get_attribute(ls8_metadata, 'FILE_NAME_BAND_' + str(band))
    
    otb_app = otbApplication.Registry.CreateApplication('BandMath')

    otb_app.SetParameterStringList('il', [ls8_band_filename])

    otb_app.SetParameterString('out', output_name)
    
    otb_app.SetParameterString('exp', '%s * im1b1 + %s' % (ls8_gain, ls8_offset))

    otb_app.ExecuteAndWriteOutput()
    
    wkt = get_wkt(output_name)
    
    metadata['identifier'] = output_name

    with open(metadata_name, 'wb') as file:
        file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        file.write(eop_metadata(metadata))    
    
    return True
    

In [126]:
def normalized_difference(band_1, band_2, metadata, suffix = ''):
    
    output_name = '%s_%s.TIF' % (get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID'),
                                                     suffix)
    metadata_name = '%s_%s.xml' % (get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID'),
                                                     suffix)
    
    otb_app = otbApplication.Registry.CreateApplication('BandMath')

    otb_app.SetParameterStringList('il', [band_1, band_2])

    otb_app.SetParameterString('out', output_name)
    
    otb_app.SetParameterString('exp', 'im1b1 >= 0 && im1b1 <= 1 && im2b1 >= 0 && im2b1 <= 1 ? ( im1b1 - im2b1 ) / ( im1b1 + im2b1 ) : 0 ')

    otb_app.ExecuteAndWriteOutput()
    
    wkt = get_wkt(output_name)
    
    metadata['identifier'] = output_name

    with open(metadata_name, 'wb') as file:
        file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        file.write(eop_metadata(metadata)) 
    
    return True
    

In [127]:
def eop_metadata(metadata):

    opt = 'http://www.opengis.net/opt/2.1'
    om  = 'http://www.opengis.net/om/2.0'
    gml = 'http://www.opengis.net/gml/3.2'
    eop = 'http://www.opengis.net/eop/2.1'
    sar = 'http://www.opengis.net/sar/2.1'
    
    root = etree.Element('{%s}EarthObservation' % opt)

    phenomenon_time = etree.SubElement(root, '{%s}phenomenonTime' % om)

    time_period = etree.SubElement(phenomenon_time, '{%s}TimePeriod' % gml)

    begin_position = etree.SubElement(time_period, '{%s}beginPosition'  % gml)

    end_position = etree.SubElement(time_period, '{%s}endPosition'  % gml)

    procedure = etree.SubElement(root, '{%s}procedure' % om)

    earth_observation_equipment = etree.SubElement(procedure, '{%s}EarthObservationEquipment' % eop)

    acquisition_parameters = etree.SubElement(earth_observation_equipment, '{%s}acquisitionParameters' % eop)

    acquisition = etree.SubElement(acquisition_parameters, '{%s}Acquisition' % sar)

    orbit_number = etree.SubElement(acquisition, '{%s}orbitNumber' % eop)

    wrs_longitude_grid = etree.SubElement(acquisition, '{%s}wrsLongitudeGrid' % eop)

    wrs_latitude_grid = etree.SubElement(acquisition, '{%s}wrsLatitudeGrid' % eop)
    
    
    feature_of_interest = etree.SubElement(root, '{%s}featureOfInterest' % om)
    footprint = etree.SubElement(feature_of_interest, '{%s}Footprint' % eop)
    multi_extentOf = etree.SubElement(footprint, '{%s}multiExtentOf' % eop)
    multi_surface = etree.SubElement(multi_extentOf, '{%s}MultiSurface' % gml)
    surface_members = etree.SubElement(multi_surface, '{%s}surfaceMembers' % gml)
    polygon = etree.SubElement(surface_members, '{%s}Polygon' % gml)    
    exterior = etree.SubElement(polygon, '{%s}exterior' % gml)  
    linear_ring = etree.SubElement(exterior, '{%s}LinearRing' % gml) 
    poslist = etree.SubElement(linear_ring, '{%s}posList' % gml) 


    result = etree.SubElement(root, '{%s}result' % om)
    earth_observation_result = etree.SubElement(result, '{%s}EarthObservationResult' % opt)
    cloud_cover_percentage = etree.SubElement(earth_observation_result, '{%s}cloudCoverPercentage' % opt)
    
    metadata_property = etree.SubElement(root, '{%s}metaDataProperty' % eop)
    earth_observation_metadata = etree.SubElement(metadata_property, '{%s}EarthObservationMetaData' % eop)
    identifier = etree.SubElement(earth_observation_metadata, '{%s}identifier' % eop)
    
    begin_position.text = metadata['startdate']
    end_position.text = metadata['enddate']
    orbit_number.text = metadata['orbitNumber']
    wrs_longitude_grid.text = metadata['wrsLongitudeGrid']
    wrs_latitude_grid.text = metadata['wrsLatitudeGrid']
    
    coords = np.asarray([t[::-1] for t in list(loads(metadata['wkt']).exterior.coords)]).tolist()
 
    pos_list = ''
    for elem in coords:
        pos_list += ' '.join(str(e) for e in elem) + ' '   

    poslist.attrib['count'] = str(len(coords))
    poslist.text = pos_list
    
    
    identifier.text = metadata['identifier'] 

    return etree.tostring(root, pretty_print=True)

In [128]:
def get_wkt(geotiff):

    src = gdal.Open(geotiff)
    
    ulx, xres, xskew, uly, yskew, yres  = src.GetGeoTransform()

    max_x = ulx + (src.RasterXSize * xres)
    min_y = uly + (src.RasterYSize * yres)
    min_x = ulx 
    max_y = uly

    source = osr.SpatialReference()
    source.ImportFromWkt(src.GetProjection())

    target = osr.SpatialReference()
    target.ImportFromEPSG(4326)

    transform = osr.CoordinateTransformation(source, target)

    result_wkt = box(transform.TransformPoint(min_x, min_y)[0],
        transform.TransformPoint(min_x, min_y)[1],
        transform.TransformPoint(max_x, max_y)[0],
        transform.TransformPoint(max_x, max_y)[1]).wkt
                
    return result_wkt           

#### Read the Landsat 8 product

In [129]:
ls8_tar = downloaded_reference
print ls8_tar

tar = tarfile.open(ls8_tar)
tar.extractall()
tar.close()

/workspace/data/LC81540332017216LGN00.tar


In [130]:
search_expression = ('*_MTL.txt')

In [131]:
ls8_metadata = glob.glob(search_expression)[0]

In [132]:
ls8_metadata

'LC08_L1TP_154033_20170804_20170812_01_T1_MTL.txt'

#### Get the metadata about the Landsat 8 product

In [133]:
ciop = cioppy.Cioppy()

ls8_catalogue_metadata = ciop.search(end_point=input_reference,
                     params=[],
                     output_fields='identifier,startdate,enddate,wkt,orbitNumber,swathIdentifier,wrsLongitudeGrid,wrsLatitudeGrid',
                     model='EOP')[0]
print ls8_catalogue_metadata

{'startdate': '2017-08-04T06:05:13.5800000Z', 'enddate': '2017-08-04T06:05:45.3500000Z', 'swathIdentifier': '', 'wrsLatitudeGrid': '33', 'orbitNumber': '', 'wrsLongitudeGrid': '154', 'identifier': 'LC08_L1TP_154033_20170804_20170812_01_T1', 'wkt': 'POLYGON((67.05402 38.23891,69.16753 37.83637,69.72306 39.55306,67.55813 39.95732,67.05402 38.23891))'}


#### Process the reflectances

In [134]:
for band in range(1,10):

    dn_to_reflectance(band, ls8_catalogue_metadata)

#### Process the vegetation indexes

In [135]:
for indice in ['NDVI', 'NDWI', 'MNDWI', 'NDBI']:
    
    if indice == 'NDVI':
        band_1 = 5
        band_2 = 4
    
    if indice == 'NDWI':
        band_1 = 3
        band_2 = 5
    
    if indice == 'MNDWI':
        band_1 = 3
        band_2 = 6
        
    if indice == 'NDBI':
        band_1 = 6
        band_2 = 5   
    
    normalized_difference('%s_R%s.TIF' % (get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID'), str(band_1)),
                          '%s_R%s.TIF' % (get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID'), str(band_2)),
                          ls8_catalogue_metadata,
                          indice)
    

#### Create the metadata file for Quality Band

In [136]:
metadata_name = '%s.xml' % (os.path.splitext(get_attribute(ls8_metadata, 'FILE_NAME_BAND_QUALITY'))[0])

print metadata_name

with open(metadata_name, 'wb') as file:
        file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        file.write(eop_metadata(ls8_catalogue_metadata)) 

LC08_L1TP_154033_20170804_20170812_01_T1_BQA.xml


#### Create the properties file for the reproducibility notebooks

In [137]:
for properties_file in ['result', 'stage-in']:

    if properties_file == 'result':
        title = 'Reproducibility notebook used for generating %s' % get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID')
    else: 
        title = 'Reproducibility stage-in notebook for Landsat8 data for generating %s' % get_attribute(ls8_metadata, 'LANDSAT_PRODUCT_ID')
        
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (ls8_catalogue_metadata['startdate'], ls8_catalogue_metadata['enddate']))
        file.write('geometry=%s' % (ls8_catalogue_metadata['wkt']))

In [138]:
metadata_name = '%s.xml' % (os.path.splitext(get_attribute(ls8_metadata, 'FILE_NAME_BAND_QUALITY'))[0])

print metadata_name

with open(metadata_name, 'wb') as file:
        file.write('<?xml version="1.0" encoding="UTF-8"?>\n')
        file.write(eop_metadata(ls8_catalogue_metadata)) 


LC08_L1TP_154033_20170804_20170812_01_T1_BQA.xml


#### Clean-up

In [139]:
for band in range(1,12):
    
    os.remove(get_attribute(ls8_metadata, 'FILE_NAME_BAND_' + str(band)))
    
for ls8_file in [get_attribute(ls8_metadata, 'ANGLE_COEFFICIENT_FILE_NAME'),
                 get_attribute(ls8_metadata, 'METADATA_FILE_NAME')]:
    
    os.remove(ls8_file)

### <a name="license">License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.